In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

In [3]:
spark = SparkSession.builder.master("local").appName("analytics").getOrCreate()

In [4]:
#4.1 Read Parquet Files
#from load step: corrected.write.mode("append").parquet("c:/sb/equity-market-data-analysis/output_dir/trade/date={}".format(date))
tradedf = spark.read.parquet("c:/sb/equity-market-data-analysis/output_dir/trade")
tradedf.createOrReplaceTempView("trades")
tradedf.show(5)

quotedf = spark.read.parquet("c:/sb/equity-market-data-analysis/output_dir/quote")
quotedf.createOrReplaceTempView("quotes")
quotedf.show(5)

+----------+------+--------+--------------------+------------+-------------------+---------+-------------------+----------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm| trade_pr|             latest|      date|
+----------+------+--------+--------------------+------------+-------------------+---------+-------------------+----------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 10:49:...|          10|2020-08-06 09:30:00|74.493774|2020-08-06 09:30:00|2020-08-06|
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:00:...|          20|2020-08-06 09:30:00|76.161194|2020-08-06 09:30:00|2020-08-06|
|2020-08-06|  SYMA|    NYSE|2020-08-06 13:11:...|          30|2020-08-06 09:30:00|76.899635|2020-08-06 09:30:00|2020-08-06|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40|2020-08-06 09:30:00| 77.11552|2020-08-06 09:30:00|2020-08-06|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:39:...|          50|2020-08-06 09:30:00| 76.37442|2020-08-06 09:30:00|2020-08-06|
+-------

In [5]:
#4.2 Create trade staging table
#4.2.1 use spark to read the trade table with date partition "2020-08-05'
query = """
    select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr from trades
    where trade_dt = '2020-08-06'
    """
df = spark.sql(query)
#4.2.2 Create a spark temporary view
df.createOrReplaceTempView("tmp_trade_moving_avg1")

In [6]:
df.show(5)

+----------+------+--------+--------------------+------------+---------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb| trade_pr|
+----------+------+--------+--------------------+------------+---------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 10:49:...|          10|74.493774|
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:00:...|          20|76.161194|
|2020-08-06|  SYMA|    NYSE|2020-08-06 13:11:...|          30|76.899635|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40| 77.11552|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:39:...|          50| 76.37442|
+----------+------+--------+--------------------+------------+---------+
only showing top 5 rows



In [7]:
#4.2.3 Calculate The 30-min Moving Average using tmp_trade_moving_avg
query = """
    SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr,
    avg(trade_pr) OVER(PARTITION BY symbol ORDER BY CAST(event_tm AS timestamp) 
        RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr
    FROM tmp_trade_moving_avg1
    """
mov_avg_df = spark.sql(query)

In [8]:
mov_avg_df.show(5)

+----------+------+--------+--------------------+------------+---------+-----------------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb| trade_pr|       mov_avg_pr|
+----------+------+--------+--------------------+------------+---------+-----------------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 10:49:...|          10|74.493774| 74.4937744140625|
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:00:...|          20|76.161194|76.16119384765625|
|2020-08-06|  SYMA|    NYSE|2020-08-06 13:11:...|          30|76.899635| 76.8996353149414|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40| 77.11552|77.11551666259766|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:39:...|          50| 76.37442|76.37442016601562|
+----------+------+--------+--------------------+------------+---------+-----------------+
only showing top 5 rows



In [9]:
#4.2.4 Save the temporary view into Hive table for staging

#mov_avg_df.write.saveAsTable("tmp_trade_moving_avg", mode='overwrite')
mov_avg_df.createOrReplaceTempView("tmp_trade_moving_avg")

In [10]:
#4.3 Create Staging Table for the prior day's last trade
#4.3.1 Get the previous date value
date = datetime.strptime('2020-08-06', '%Y-%m-%d')
print(date)
prev_date_str = str(date.date() - timedelta(days=1))
print(prev_date_str)

#4.3.2 Use Spark to read the trade table with date partition prev_date_str 
query = """
    select trade_dt, symbol, exchange, event_tm, event_seq_nb, trade_pr from trades
    where trade_dt = '{}'
    """
df = spark.sql(query.format(prev_date_str))
df.show(5)

2020-08-06 00:00:00
2020-08-05
+----------+------+--------+--------------------+------------+---------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb| trade_pr|
+----------+------+--------+--------------------+------------+---------+
|2020-08-05|  SYMA|    NYSE|2020-08-05 10:37:...|          10|79.194885|
|2020-08-05|  SYMA|    NYSE|2020-08-05 11:56:...|          20| 76.49093|
|2020-08-05|  SYMA|    NYSE|2020-08-05 13:09:...|          30| 75.04523|
|2020-08-05|  SYMA|    NYSE|2020-08-05 14:24:...|          40|78.430954|
|2020-08-05|  SYMA|    NYSE|2020-08-05 15:31:...|          50| 78.15297|
+----------+------+--------+--------------------+------------+---------+
only showing top 5 rows



In [11]:
#4.3.3 Create spark temporary view
df.createOrReplaceTempView("tmp_last_trade1")

In [12]:
#4.3.4 Calculate last trade price using tmp_last_trade
query = """
    SELECT symbol, exchange, trade_pr AS close_pr
    FROM tmp_last_trade1 t1
    JOIN
    (SELECT MAX(event_tm) AS last_record FROM tmp_last_trade1
     GROUP BY symbol, exchange) t2
    ON t1.event_tm == last_record
    """

In [13]:
last_pr_df = spark.sql(query)

In [14]:
last_pr_df.show(5)

+------+--------+---------+
|symbol|exchange| close_pr|
+------+--------+---------+
|  SYMA|    NYSE| 77.78611|
|  SYMB|    NYSE|33.956287|
|  SYMC|    NYSE|160.61949|
+------+--------+---------+



In [15]:
last_pr_df.createOrReplaceTempView("tmp_last_trade")

In [16]:
#4.4.1.2 Create spark temp view to union both tables
quote_union = spark.sql("""
    SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb, NULL as arrival_tm, NULL as bid_size, 
        NULL as ask_pr, NULL as ask_size, NULL as latest, trade_pr, mov_avg_pr 
    FROM tmp_trade_moving_avg
    UNION 
    SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb, arrival_tm, bid_size, ask_pr, ask_size,
        latest, NULL as trade_pr, NULL as mov_avg_pr 
    FROM quotes
    """)
quote_union.createOrReplaceTempView("quote_union")
quote_union.show(10)

+----------+------+--------+--------------------+------------+----------+--------+------+--------+------+---------+-----------------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|arrival_tm|bid_size|ask_pr|ask_size|latest| trade_pr|       mov_avg_pr|
+----------+------+--------+--------------------+------------+----------+--------+------+--------+------+---------+-----------------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 22:00:...|         100|      null|    null|  null|    null|  null| 76.30847| 76.3084716796875|
|2020-08-06|  SYMA|    NYSE|2020-08-06 14:27:...|          40|      null|    null|  null|    null|  null| 77.11552|77.11551666259766|
|2020-08-06|  SYMA|    NYSE|2020-08-06 13:11:...|          30|      null|    null|  null|    null|  null|76.899635| 76.8996353149414|
|2020-08-06|  SYMA|    NYSE|2020-08-06 16:58:...|          60|      null|    null|  null|    null|  null| 78.32364|78.32363891601562|
|2020-08-06|  SYMA|    NYSE|2020-08-06 20:49:...|          90|

In [17]:
#4.4 Populate the latest trade_pr and mov_avg_pr 
quote_union_update = spark.sql("""
SELECT quote_union.trade_dt, quote_union.symbol, quote_union.exchange, quote_union.event_tm, 
        quote_union.event_seq_nb, quote_union.arrival_tm, quote_union.bid_size, quote_union.ask_pr, 
        quote_union.ask_size, quote_union.latest, latest_trade.latest_trade_pr, latest_trade.mov_avg_pr
FROM quote_union
JOIN (SELECT quote.symbol, quote.exchange, quote.trade_pr as latest_trade_pr, quote.mov_avg_pr
    FROM quote_union quote
    JOIN (SELECT MAX(event_tm) as latest_event_tm FROM quote_union GROUP BY symbol, exchange) last_trade_tm
        ON quote.event_tm == last_trade_tm.latest_event_tm) latest_trade
    ON quote_union.symbol == latest_trade.symbol AND quote_union.exchange == latest_trade.exchange
    """)
quote_union_update.createOrReplaceTempView("quote_union_update")
quote_union_update.show(10)

+----------+------+--------+--------------------+------------+-------------------+--------+--------+--------+-------------------+---------------+----------------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm|bid_size|  ask_pr|ask_size|             latest|latest_trade_pr|      mov_avg_pr|
+----------+------+--------+--------------------+------------+-------------------+--------+--------+--------+-------------------+---------------+----------------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:52:...|          27|2020-08-06 09:30:00|     100|78.24039|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:12:...|          46|2020-08-06 09:30:00|     100| 78.0638|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|
|2020-08-06|  SYMA|    NYSE|2020-08-06 18:37:...|          73|2020-08-06 09:30:00|     100|76.86819|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|
|2020-08-06|  SYMA|   

In [18]:
#4.4.1.4 Filter for quote records
quote_update = spark.sql("""
SELECT *
FROM quote_union_update
WHERE ask_size IS NOT NULL
""")
quote_update.createOrReplaceTempView("quote_update")
quote_update.show(5)

+----------+------+--------+--------------------+------------+-------------------+--------+--------+--------+-------------------+---------------+----------------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm|bid_size|  ask_pr|ask_size|             latest|latest_trade_pr|      mov_avg_pr|
+----------+------+--------+--------------------+------------+-------------------+--------+--------+--------+-------------------+---------------+----------------+
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:52:...|          27|2020-08-06 09:30:00|     100|78.24039|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|
|2020-08-06|  SYMA|    NYSE|2020-08-06 15:12:...|          46|2020-08-06 09:30:00|     100| 78.0638|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|
|2020-08-06|  SYMA|    NYSE|2020-08-06 18:37:...|          73|2020-08-06 09:30:00|     100|76.86819|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|
|2020-08-06|  SYMA|   

In [19]:
#4.4.2 Join with table temp_last_trade to get the prior day close price
quote_final = spark.sql("""
SELECT * 
FROM quote_update 
LEFT JOIN tmp_last_trade
    USING(symbol, exchange)
""")
quote_final.show(5)

+------+--------+----------+--------------------+------------+-------------------+--------+--------+--------+-------------------+---------------+----------------+--------+
|symbol|exchange|  trade_dt|            event_tm|event_seq_nb|         arrival_tm|bid_size|  ask_pr|ask_size|             latest|latest_trade_pr|      mov_avg_pr|close_pr|
+------+--------+----------+--------------------+------------+-------------------+--------+--------+--------+-------------------+---------------+----------------+--------+
|  SYMA|    NYSE|2020-08-06|2020-08-06 12:52:...|          27|2020-08-06 09:30:00|     100|78.24039|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|77.78611|
|  SYMA|    NYSE|2020-08-06|2020-08-06 15:12:...|          46|2020-08-06 09:30:00|     100| 78.0638|     100|2020-08-06 09:30:00|       76.30847|76.3084716796875|77.78611|
|  SYMA|    NYSE|2020-08-06|2020-08-06 18:37:...|          73|2020-08-06 09:30:00|     100|76.86819|     100|2020-08-06 09:30:00|       76.3

In [20]:
#4.4.3 Write the final dataframe into Azure Blob storage at corresponding partition
#temp local storage for jupyter notebook
quote_final.write.parquet("c:/sb/equity-market-data-analysis/output_dir/quote-trade-analytical/date=2020-08-06")